# Time Scattering in Kymatio

In this tutorial we will work through examples of time scattering (Scattering1D) in Kymatio

* theory of the wavelet scattering transform of multiple orders
* understanding the scattering filterbank construction parameters (J, Q, order)
* visualizing the filters
* visualizing the response to modulated sounds that appear in music and speech
    - amplitude modulation (tremolo)
    - frequency modulation (vibrato)
    - attacks (note onset)
    - amplitude modulation interference 
* generate a dataset of synthetic signals: varying spectral shape and interference patterns
* unsupervised manifold embedding of the nearest neighbour graph (Isomap) of the dataset under Scattering1D, MFCCs and Open-L3 deep features

[Kymatio Github](https://github.com/kymatio/kymatio)

- explain J, Q
- check out xi at each order
- plot the filters at each order
- explain the transform, averaging
- show invariance to time shifts, dilations
- visualize non-averaged 1O, averaged 1O, scat 2O

### Installation

In [ ]:
!pip install kymatio

#### Import Scattering1D 
- Let's import the torch frontend. Tensorflow, Numpy and Jax are also available.

In [3]:
import numpy as np, torch
import matplotlib.pyplot as plt
from kymatio.torch import Scattering1D
from kymatio.scattering1d.filter_bank import scattering_filter_factory

#### Generate a Scattering Filterbank

First, we will use `scattering_filter_factory` to inspect the construction of a scattering filterbank

To construct the filterbank, we define the following parameters: 
- the temporal support of the filters $T$, which corresponds to the size of the input signal and must be a power of two
- the maximum scale $J$. The lowest scale (largest filter, lowest central frequency) will be defined over a time support of $2^J$. $J$ also determines the total number of octaves.
- the number of wavelets filters per octave $Q$. When $Q = 1$, we get a dyadic wavelet filterbank i.e. subsequent wavelet filter central frequencies are spaced by a factor two. $Q$ determines how well we can localize a signal in frequency

In [5]:
help(scattering_filter_factory)

Help on function scattering_filter_factory in module kymatio.scattering1d.filter_bank:

scattering_filter_factory(J_support, J_scattering, Q, T, r_psi=0.7071067811865476, criterion_amplitude=0.001, normalize='l1', max_subsampling=None, sigma0=0.1, alpha=4.0, P_max=5, eps=1e-07, **kwargs)
    Builds in Fourier the Morlet filters used for the scattering transform.
    
    Each single filter is provided as a dictionary with the following keys:
    * 'xi': central frequency, defaults to 0 for low-pass filters.
    * 'sigma': frequential width
    * k where k is an integer bounded below by 0. The maximal value for k
        depends on the type of filter, it is dynamically chosen depending
        on max_subsampling and the characteristics of the filters.
        Each value for k is an array (or tensor) of size 2**(J_support - k)
        containing the Fourier transform of the filter after subsampling by
        2**k
    * 'width': temporal width (interval of temporal invariance, i.e. its "